In [2]:
import sys
sys.path.append("../..") 

from custom_scripts import database

In [18]:
### Table of unique carrier identifiers present in the test samples
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)
carriers.head(1)

,op_unique_carrier,count
0,9E,23088


In [19]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)
test_flight_numbers.head(1)

,op_carrier_fl_num,count
0,6114,64


In [20]:
### Table of flight numbers present in the samples available for training, and a count of their frequency
past_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*)
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
past_flight_numbers.to_csv('../../data/preprocessing/past_flight_numbers.csv', index=False)
past_flight_numbers.head(1)


,op_carrier_fl_num,count
0,1,308


In [28]:
### Historic average delay times by flight number
avg_flight_delays_by_flight_number = database.query("""
                                SELECT  op_carrier_fl_num,
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(taxi_in) AS "avg_taxi_in", 
                                        AVG(taxi_out) AS "avg_taxi_out", 
                                        AVG(wheels_off) AS "avg_wheels_off", 
                                        AVG(wheels_on) AS "avg_wheels_on",
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
avg_flight_delays_by_flight_number.to_csv('../../data/preprocessing/average_flight_delays_by_flight_number.csv', index=False)
avg_flight_delays_by_flight_number.head(1)


,op_carrier_fl_num,avg_arr_delay,avg_taxi_in,avg_taxi_out,avg_wheels_off,avg_wheels_on,avg_late_aircraft_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_dep_delay
0,1,-10.616883,8.727273,17.769481,836.483766,1058.516234,3.941176,29.794118,4.382353,14.529412,2.918831


In [29]:
### Average flight delays grouped by destination airport ID
avg_flight_delays_by_destination_airport = database.query("""
                                SELECT  dest_airport_id, 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(taxi_in) AS "avg_taxi_in", 
                                        AVG(taxi_out) AS "avg_taxi_out", 
                                        AVG(wheels_off) AS "avg_wheels_off", 
                                        AVG(wheels_on) AS "avg_wheels_on",
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY dest_airport_id;
                                """)
avg_flight_delays_by_destination_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_destination_airport.csv', index=False)
avg_flight_delays_by_destination_airport.head(1)

,dest_airport_id,avg_arr_delay,avg_taxi_in,avg_taxi_out,avg_wheels_off,avg_wheels_on,avg_late_aircraft_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_dep_delay
0,10135,6.524297,4.895141,21.410486,1581.957801,1682.02046,26.819767,21.19186,6.401163,9.598837,11.323529


In [3]:
### Average flight delays grouped by origin airport ID
avg_flight_delays_by_origin_airport = database.query("""
                                SELECT  origin_airport_id, 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(taxi_in) AS "avg_taxi_in", 
                                        AVG(taxi_out) AS "avg_taxi_out", 
                                        AVG(wheels_off) AS "avg_wheels_off", 
                                        AVG(wheels_on) AS "avg_wheels_on",
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY origin_airport_id;
                                """)
avg_flight_delays_by_origin_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_origin_airport.csv', index=False)
avg_flight_delays_by_origin_airport.head(1)

,origin_airport_id,avg_arr_delay,avg_taxi_in,avg_taxi_out,avg_wheels_off,avg_wheels_on,avg_late_aircraft_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_dep_delay
0,10135,9.233933,10.689345,18.04878,1233.286264,1364.227214,28.378698,29.207101,2.35503,19.940828,12.514763
